# A Short MACE Tutorial

## Introduction

This is a short tutorial for MACE, a highly accurate and efficient ML interatomic potential.
Please read the associated [paper](https://arxiv.org/pdf/2206.07697.pdf).
The reference implementation is available [here](https://github.com/ACEsuit/mace).

## Installation

In [ ]:
# Install dependencies
!pip install e3nn==0.4.4 opt_einsum ase torch_ema prettytable

# Clone MACE
!git clone --depth 1 https://github.com/ACEsuit/mace.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.7/387.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 27.9 MB/s eta 0:00:00
Cloning into 'mace'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 104 (delta 3), reused 28 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (104/104), 51.01 MiB | 19.20 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [ ]:
!pip install mace/

Processing ./mace
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.8/448.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.4/891.4 kB 31.0 MB/s eta 0:00:00
  Created wheel for mace-torch: filename=mace_torch-0.3.7-py3-none-any.whl size=139209 sha256=9ed574d8c89831aa79f02aba592076a98518fc3f5078cb04fb56b11e6c73a9d4
  Stored in directory: /tmp/pip-ephem-wheel-cache-a8lm5h6p/wheels/df/5f/32/ef59561725170a81c728fd01c75e56a9ee83bad6da485fc6a5
  Created wheel for python-hostlist: filename=python_hostlist-2.0.0-py3-none-any.whl size=39465 sha256=3affcf62841bf4f73fe902b74d3400f56a27ab4a5af2bf925ecb695f41b171e9
  Stored in directory: /root/.cache/pip/wheels/71/e9/0b/1e7a2ca15b702dd36c81c33dad4c23773270dc417eca48dc72
Successfully built mace-torch python-hostlist


**Note:** Make sure to enable GPU: Runtime --> Change runtime type to GPU

## Loading Data
The data files used to train the MACE model have to be in `extxyz` format.
In this tutorial, we use the 3BPA dataset consisting of 500 configurations sampled a 300K with DFT.
The energies are in eV and forces in eV/A.

In [ ]:
!git clone https://github.com/davkovacs/BOTNet-datasets.git

Cloning into 'BOTNet-datasets'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 57 (delta 13), reused 37 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (57/57), 28.73 MiB | 17.47 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [ ]:
!ls BOTNet-datasets/dataset_3BPA

iso_atoms.xyz  test_1200K.xyz  test_600K.xyz  train_300K.xyz
README.md      test_300K.xyz   test_dih.xyz   train_mixedT.xyz


## Training

To train a MACE model you can specify the training file with the `--train_file` flag. The validation set can either be specified as a separate file using the `--valid_file` keyword, or it can be specified as a fraction of the training set using the `--valid_fraction` keyword. It is also possible to provide a test set that only gets evaluated at the end of the training using the `--test_file` keyword. If you want to compute the RMSE for different parts of the training set separately, specify the `config_type` keyword in the `info` dict of the configurations.

When parsing the data files the energies are read using the keyword `energy` and the forces using the keyword `forces`. To change that, specify the `--energy_key` and `--forces_key`.

For illustration, we create a very small model with 16 invariant messages specified by `hidden_irreps='16x0e'`.

In [ ]:
!python3 ./mace/scripts/run_train.py \
  --name="MACE_model" \
  --train_file="BOTNet-datasets/dataset_3BPA/train_300K.xyz" \
  --valid_fraction=0.05 \
  --forces_key="forces" \
  --energy_key="energy" \
  --test_file="BOTNet-datasets/dataset_3BPA/test_300K.xyz" \
  --E0s='{1:-13.663181292231226, 6:-1029.2809654211628, 7:-1484.1187695035828, 8:-2042.0330099956639}' \
  --model="ScaleShiftMACE" \
  --hidden_irreps='32x0e' \
  --r_max=4.0 \
  --batch_size=20 \
  --max_num_epochs=100 \
  --ema \
  --ema_decay=0.99 \
  --amsgrad \
  --default_dtype="float32" \
  --device=cpu \
  --seed=123 \
  --swa

/usr/local/lib/python3.10/dist-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_indices = torch.load(os.path.join(os.path.d

It is possible to use `--model=MACE`, in order to have the correct limit for isolated atoms. This is recommanded for task studying bond breaking events.

## Run

The trained model is realidy usable to run some ASE MD for illustration. The Colab hardware are not very performant so we put a small number of timesteps for illustration.

In [ ]:
from ase import units
from ase.md.langevin import Langevin
from ase.io import read, write
import numpy as np
import time

from mace.calculators import MACECalculator

calculator = MACECalculator(model_paths='/content/checkpoints/MACE_model_run-123.model', device='cpu')
init_conf = read('BOTNet-datasets/dataset_3BPA/test_300K.xyz', '0')
init_conf.set_calculator(calculator)

dyn = Langevin(init_conf, 0.5*units.fs, temperature_K=310, friction=5e-3)
def write_frame():
        dyn.atoms.write('md_3bpa.xyz', append=True)
dyn.attach(write_frame, interval=50)
dyn.run(100)
print("MD finished!")

/usr/local/lib/python3.10/dist-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_indices = torch.load(os.path.join(os.path.d

No dtype selected, switching to float32 to match model dtype.


<ipython-input-6-7dfb3a8dbc2a>:11: DeprecationWarning: Please use atoms.calc = calc
  init_conf.set_calculator(calculator)


KeyboardInterrupt: 